<a href="https://colab.research.google.com/github/Frosti385/Programmierprojekt/blob/developLars/programmierprojekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import torch
import torchvision
from torch.optim import lr_scheduler
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import make_grid
import numpy as np


def load_data(path: str):
    with open(path, "r") as f:
        files = f.readlines()
        targets = []
        images = []

        for line in files:
            name, target = line.split(" ")
            if name.startswith("paper"):
                path = f"/content/drive/MyDrive/data/paper/{name}"
            elif name.startswith("glass"):
                path = f"/content/drive/MyDrive/data/glass/{name}"
            elif name.startswith("metal"):
                path = f"/content/drive/MyDrive/data/metal/{name}"
            elif name.startswith("cardboard"):
                path = f"/content/drive/MyDrive/data/cardboard/{name}"
            elif name.startswith("plastic"):
                path = f"/content/drive/MyDrive/data/plastic/{name}"
            elif name.startswith("trash"):
                path = f"/content/drive/MyDrive/data/trash/{name}"

            targets.append(int(target)-1)
            image = torchvision.io.read_image(path)
            images.append(image)

    return images, targets



class TrashDataset(Dataset):

  def __init__(self, images, targets):
    self.targets = targets
    self.images = images

  def __len__(self):
    return len(self.targets)

  transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.Resize((224, 224)),
     transforms.ToTensor()])

  def __getitem__(self, index):
    return self.targets[index], self.transform(self.images[index])


class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.network = torchvision.models.vgg19(pretrained=True)
    self.fc1 = nn.Linear(1000, 6)
  def forward(self, x):
    x = self.network(x)
    x = self.fc1(x)
    return x


In [ ]:
import os
import torch
import torchvision
from torch.optim import lr_scheduler
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import make_grid
import numpy as np


def load_data(path: str):
    with open(path, "r") as f:
        files = f.readlines()
        targets = []
        images = []

        for line in files:
            name, target = line.split(" ")
            if name.startswith("paper"):
                path = f"/content/drive/MyDrive/data/paper/{name}"
            elif name.startswith("glass"):
                path = f"/content/drive/MyDrive/data/glass/{name}"
            elif name.startswith("metal"):
                path = f"/content/drive/MyDrive/data/metal/{name}"
            elif name.startswith("cardboard"):
                path = f"/content/drive/MyDrive/data/cardboard/{name}"
            elif name.startswith("plastic"):
                path = f"/content/drive/MyDrive/data/plastic/{name}"
            elif name.startswith("trash"):
                path = f"/content/drive/MyDrive/data/trash/{name}"

            targets.append(int(target)-1)
            image = torchvision.io.read_image(path)
            images.append(image)

    return images, targets


class TrashDataset(Dataset):

  def __init__(self, images, targets):
    self.targets = targets
    self.images = images

  def __len__(self):
    return len(self.targets)

  transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.Resize((224, 224)),
     transforms.ToTensor()])

  def __getitem__(self, index):
    return self.targets[index], self.transform(self.images[index])


class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.network = torchvision.models.vgg16(pretrained=True)
    self.fc1 = nn.Linear(1000, 6)
  def forward(self, x):
    x = self.network(x)
    x = self.fc1(x)
    return x


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batch_size = 40
cv_scores = []

train_images, train_targets = load_data("/content/drive/MyDrive/garbage_files/one-indexed-files-notrash_train.txt")
train_dataset = TrashDataset(images=train_images, targets=train_targets)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

cv_images, cv_targets = load_data("/content/drive/MyDrive/garbage_files/one-indexed-files-notrash_val.txt")
cv_dataset = TrashDataset(images=cv_images, targets=cv_targets)
cv_loader = torch.utils.data.DataLoader(cv_dataset, batch_size=batch_size, shuffle=True, num_workers=0)


net = Net()
net.to(device)

#num_ftrs = 512*7*7
#model_ft.fc = nn.Linear(num_ftrs, 2)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

for epoch in range(20):
    for idx, data in enumerate(train_loader):
        targets, inputs = data
        targets, inputs = targets.cuda(), inputs.cuda()

        optimizer_ft.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer_ft.step()
        print(f'[{epoch + 1}, {idx + 1:5d}] loss: {loss}')

        correct = 0
        total = 0
    with torch.no_grad():
        for data in cv_loader:
            targets, inputs = data
            targets, inputs = targets.cuda(), inputs.cuda()
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

        print(f'Accuracy of the network on the cv images: {100 * correct // total} %')
        
        torch.save(net, "/content/drive/MyDrive/models/epoch.max")

        cv_scores.append(100 * correct // total)



    
print('Finished Training')
print(cv_scores)


[1,     1] loss: 2.4212141036987305
[1,     2] loss: 1.8982009887695312
[1,     3] loss: 1.449749231338501
[1,     4] loss: 1.917091965675354
[1,     5] loss: 1.7124236822128296
[1,     6] loss: 1.709467887878418
[1,     7] loss: 1.4384715557098389
[1,     8] loss: 1.107297420501709
[1,     9] loss: 1.3704907894134521
[1,    10] loss: 1.463112711906433
[1,    11] loss: 1.3490612506866455
[1,    12] loss: 0.896816074848175
[1,    13] loss: 1.2504932880401611
[1,    14] loss: 1.275001883506775
[1,    15] loss: 1.0441551208496094
[1,    16] loss: 0.6955261826515198
[1,    17] loss: 0.6315150260925293
[1,    18] loss: 1.397944450378418
[1,    19] loss: 1.07841157913208
[1,    20] loss: 1.2193033695220947
[1,    21] loss: 1.1354093551635742
[1,    22] loss: 0.7114309668540955
[1,    23] loss: 1.0083913803100586
[1,    24] loss: 1.053147554397583
[1,    25] loss: 0.9105925559997559
[1,    26] loss: 0.8371565937995911
[1,    27] loss: 0.8187342882156372
[1,    28] loss: 0.8142145872116089
[1,

In [ ]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

classes = ['Glas', 'Papier', 'Pappe', 'Plastik', 'Metall', 'Müll']
batch_size = 64
net = Net()
net.to(device)

model = torch.load("/content/drive/MyDrive/models/epoch.max")
try_image, try_target = load_data("/content/drive/MyDrive/garbage_files/one-indexed-files-notrash_test.txt")
try_dataset = TrashDataset(images=try_image, targets=try_target)
try_loader = torch.utils.data.DataLoader(try_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

"""
for data in try_loader:
  targets, inputs = data
  #targets, inputs = targets.cuda(), inputs.cuda()
  outputs = net(inputs)
  _, predicted = torch.max(outputs.data,1)
  print(f"Der Müll auf dem Bild ist {classes[predicted]}")
"""
model.eval()
correct = 0
total = 0

with torch.no_grad():
  for data in try_loader:
    targets, inputs = data
    targets, inputs = targets.cuda(), inputs.cuda()
    # calculate outputs by running images through the network
    outputs = model(inputs)
    # the class with the highest energy is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += targets.size(0)
    correct += (predicted == targets).sum().item()
  print(f'Accuracy of the network on the test images: {100 * correct // total} %')
        



Accuracy of the network on the test images: 88 %
